In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys

# TODO: change this to the path to your homework folder
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'NLP /Final project/ChineseHumorSentiment-master/data'
# GOOGLE_DRIVE_PATH = os.path.join('drive', 'Shared drives', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
# print(os.listdir(GOOGLE_DRIVE_PATH))
# sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
import pandas as pd

In [ ]:
train_set = pd.read_csv("drive/Shared drives/NLP Final Project/Dataset/balanced_train_task2.csv",encoding='utf8')
test_set = pd.read_csv("drive/Shared drives/NLP Final Project/Dataset/balanced_test_task2.csv",encoding='utf8')
train_set = train_set.drop(columns=['id', 'Unnamed: 0'])
test_set = test_set.drop(columns=['id', 'Unnamed: 0'])

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Preprocessing

In [ ]:
! pip install transformers

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 35.9 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 47.3 MB/s 
     |████████████████████████████████| 6.5 MB 29.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
train_set

,joke,label
0,请告诉我，史密斯先生，面试官问道，您还有什么其他您认为值得一提的技能吗？ 的确还有，应聘者谦...,1
1,一辆小汽车急速闯过红灯，刚好被警察拦住。 喂，你难道没有看见红灯吗？ 不，真对不起，我看到红...,1
2,教官子与县丞子厮打，教官子屡负，归而哭诉其母。母 曰：彼家终日吃肉，故恁般强健会打。你家终日...,0
3,经验是我们秃顶后大自然赋予我们的梳子。,1
4,正在进行中 - 它可能很慢，但它很稳定。请耐心等待。谢谢。,1
...,...,...
8995,一官员因贪污受贿被判刑，儿子探监时，说自己大学毕业找不到工作。 官员说：会有人来帮你的，我在...,1
8996,某医院急诊室送来一位病人，医生见病人痛得厉害，就给他服了一片止痛药，可是病人仍疼痛不止。 家...,2
8997,小学时候，每天出发前爸爸总会躺在床上提醒我： 校徽，红领巾，书包......天天被他说烦了，...,2
8998,法官问犯人：“你一点法律意识没有怎么不去学点法律知识啊？” 犯人答：“你当我傻啊！知法犯法―...,2


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

example_text = train_set.iloc[400, 0]
bert_input = tokenizer(example_text,padding='max_length', max_length = 40, 
                       truncation=True, return_tensors="pt")

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [ ]:
print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

tensor([[ 101, 1963, 3362, 1728, 3336, 5945, 3172,  772, 1501, 6574, 7030, 7309,
         7579, 2193, 5636, 6912, 2097, 1927, 6571, 8024, 1322, 2157, 2199, 2824,
         2857, 2111, 2094, 8123, 2259,  722, 1184, 4638, 1825, 3315, 4495, 3833,
         6589,  511, 2496,  102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] 如 果 因 杜 蕾 斯 产 品 质 量 问 题 导 致 避 孕 失 败 ， 厂 家 将 承 担 孩 子 18 岁 之 前 的 基 本 生 活 费 。 当 [SEP]


In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
labels = {'true': 1,
          'false': 0
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [label for label in df['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 40, truncation=True,
                                return_tensors="pt") for text in df['joke']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-chinese')
        # self.dropout = nn.Dropout(dropout)
        self.rnn = nn.LSTM(768, 200, num_layers = 3, device='cuda', batch_first=False, bidirectional=True)
        self.linear = nn.Linear(400, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # dropout_output = self.dropout(pooled_output)
        rnn_output, _ = self.rnn(pooled_output.view(len(pooled_output), 1, -1))
        linear_output = self.linear(rnn_output)
        final_layer = (linear_output).reshape((2, 3))
        return final_layer

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
              
train(model, train_set, test_set, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 4500/4500 [10:10<00:00,  7.37it/s]


Epochs: 1 | Train Loss:  0.547                 | Train Accuracy:  0.425                 | Val Loss:  0.543                 | Val Accuracy:  0.467


100%|██████████| 4500/4500 [10:09<00:00,  7.38it/s]


Epochs: 2 | Train Loss:  0.534                 | Train Accuracy:  0.510                 | Val Loss:  0.527                 | Val Accuracy:  0.492


 28%|██▊       | 1257/4500 [02:50<07:19,  7.38it/s]

In [ ]:
torch.save(model, "drive/Shared drives/NLP Final Project/Star_model/three_way_bert")